# Этап А. Обкачивание Краулером сайт, создание БД и морф обработка

## Часть 1. Краулер и БД

Для того чтобы что-то сделать с анекдотами их сначала надо собрать. Куда? В базу данных. Откуда? С сайта.
1. Создать бд
2. Рассмотреть исходный код сайта
3. Скраулерить оттуда инфу в бд 

### 1.1. Создадим БД

Архитектура: 3 таблицы, 2 из них с информацией, одна - связующая
* **anecs** - сами анекдоты, лайки, год выхода
* **tags** - сюда я вытащю именованные сущности (и не только) в каких-либо видах
* **text_to_tag** - само индексирование = связь анекдотов и тегов в них

In [13]:
import sqlite3
from bs4 import BeautifulSoup
import re

In [14]:
conn = sqlite3.connect('le_petit_bd.db')
cur = conn.cursor()

In [15]:
cur.execute("""
CREATE TABLE IF NOT EXISTS anecs 
(id INTEGER PRIMARY KEY AUTOINCREMENT, text text, lemtext text, time int, likes int, kwords text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS tags 
(id_tag int PRIMARY KEY, auto_tag text, my_tag text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS text_to_tag 
(id INTEGER PRIMARY KEY AUTOINCREMENT, id_anec int, id_tag int) 
""")


conn.commit()
conn.close()

Сайт, с которым я работаю: https://www.anekdot.ru/release/anekdot/year/2022/42
Обкачаю топ 20 страниц (ранжированных по популярности) анекдотов за последние 3 года.

### 1.2. Краулером обкачаем анекдотики

In [21]:
import requests
from pprint import pprint
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [ ]:
# Пусть тут пока побудет код разовых вызовов, где я тренировалась

In [49]:
full_url = 'https://www.anekdot.ru/release/anekdot/year/2022/33'
req = session.get(full_url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')

#a = soup.find_all('div', {'class': 'text'}) 
#print(a[3].text)

b = soup.find_all('div', {'class': 'rates'}) 
# Регулярными выражениями достанем кол-во лайков
c = re.search(r'r="(\d+);', str(b[0]))
c.group(0) # Чё-то странным способом, почему скобки не работают? Ну лан
d = re.search(r'\d+', c.group(0))
d

<re.Match object; span=(3, 5), match='14'>

**Напишем сразу функцию, которая будет делать следующее:**  
   1. С полученной страницы доставать все анекдоты и их рейтинг (в значении оценки)   
   2. Проверять анекдоты на длину {для красоты и удобства бота я решила не брать слишком длинные}    
   3. В случае если длина нормальная - записывает в базу данных  
        *а) текст анекдота   
         б) его оценки  
         в) год публикации этого анекдота*    
**P.S.** Отмечу, что на самом сайте изредка попадаются местные теги, включающие в себя и именованные сущности, но они так редко стоят, что я решила нет смысла трогать их, раз всё равно картину полностью они не отражают => не юзанёшь.

In [74]:
def write_all_to_db(year, number_of_page):
    full_url = 'https://www.anekdot.ru/release/anekdot/year/' + str(year) + "/" + str(number_of_page)
    req = session.get(full_url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    conn = sqlite3.connect('le_petit_bd.db')
    cur = conn.cursor()
    
    all_anecs = soup.find_all('div', {'class': 'text'}) 
    all_likes = soup.find_all('div', {'class': 'rates'})
    
    for i in range(len(all_anecs)-1): # Потому что тут ещё Анекдот дня лишний
        anecdot = all_anecs[i].text
        promezutoc = re.search(r'r="(\d+);', str(all_likes[i]))
        likes = int(re.search(r'\d+', promezutoc.group(0)).group(0))
        
        if len(anecdot) < 638: # Не хочу в бот слишком длинные анекдоты, не красиво
            cur.execute(
                """
                INSERT INTO anecs 
                    (text, time, likes)
                    VALUES (?, ?, ?)
                """, (
                    anecdot, year, likes)
            )
            conn.commit()
    conn.close()

In [75]:
from tqdm.auto import tqdm # Пусть грузится, пока буду кушать

И циклом запишу сразу в базу данных.  
P.S. Без проверки на анекдоты-повторки, пушто мой метод сбора данных исключает повторки (так как по 3 разным годам), да ив  принципе повторки не страшны 

In [76]:

years = [2022, 2021, 2020]
for i in years:
    year = i
    print("Год", i, "начат.")
    for i in tqdm(range(20)):
        try:
            write_all_to_db(year, i+1)
        except Exception as e:
            print('full exception', e)  # Конечно же с проверкой


Год 2022 начат.



Год 2021 начат.



Год 2020 начат.


Собственно, ура, у нас собрано 5963 анекдота, круто же

## Часть 2. Морфанализатор анекдотов

Как бы что делать дальше? Дальше взять тексты анекдотов (вы-та-щить) и обработать их.   
Для простейшего поиска вхождений тегов достаточно лемматизировать, но, с верой в потенциальное развитие чего-нибудь, очистим и от стоп слов и от пунктуации. Полученные анекдоты запишем а БД.
1. Функция лемматизации (полной обработки) 
2. Вытащить анекдоты из Бд
3. Обработать
4. Вписать обратно

### 2.1. Напишем функцию лемматизации

In [84]:
from pymystem3 import Mystem

from nltk.tokenize import wordpunct_tokenize

In [186]:
m = Mystem()
stops = set(stopwords.words('russian'))

In [117]:
# Это я тренировалась что-то сделать
text_a = "Россия в очередной раз пожертвовала собой. Остановила мировую ковидную истерию."

In [187]:
def my_lemmatize(x):
        if type(x) != str:
            return "" # Ну на всякий
        x = x.lower()
        lemmas = m.lemmatize(x)
        result = []
        for word in lemmas:
            if word not in stops and word != " ":
                result.append(word)
        return " ".join(result)

In [119]:
my_lemmatize(text_a)

'россия очередной пожертвовать . останавливать мировой ковидный истерия . \n'

### 2.2. Вытащим в один лист все наши анекдоты собранные

In [153]:
con = sqlite3.connect('le_petit_bd.db') # пока на другой (не тот и не та)
cur = con.cursor()

In [154]:
comedy_query = """
SELECT anecs.id, anecs.text FROM anecs
"""

In [181]:
cur.execute(comedy_query, )
longlist = cur.fetchall()
longlist[0]

(1,
 'Впервые за 28 лет депутатам придётся жить, лечиться, учить своих детей в условиях, которые они все эти годы создавали.')

Что мы имеем на данный момент? Список, состоящий из туплов, на нулевой позиции в которых индекс текста, а на первой текст анекдотика (5963 кортежей)

### 2.3. Лемматизируем каждый анекдот 

Напишем функцию с переводом в нужный для записи в БД формат.

In [188]:
my_lemmatize(longlist[0][1])

'впервые 28 год депутат приходиться жить ,  лечиться ,  учить свой ребенок условие ,  который весь год создавать . \n'

In [189]:
def tuple_to_tuple(x):
    text_no_lemmas = x[1]
    text_of_lemmas = my_lemmatize(text_no_lemmas)
    return tuple((x[0], text_of_lemmas))

In [190]:
new_longlist = []
for i in tqdm(range(len(longlist))):
    q = longlist[i]
    new_longlist.append(tuple_to_tuple(q))

In [219]:
new_longlist[1]

(2,
 '-  представлять ,  товарищ жук ,  сегодня сон присниться , -  говорить сталин , -  москва минск нападывать киев ,  канцлер германия умолять война прекращать ! \n')

Я сглупила и не убрала знаки препинания (на случай если решу привязывать моделеподобное что-то всё-таки)

In [220]:
# Пишем функцию
def no_punctuation(a):
    a = wordpunct_tokenize(a)
    b = ""
    for i in a:
        if i not in ",.:;!?\/&#@-":
            b += i
            b += " "
    return b.rstrip()


In [221]:
# Применяем её на нашем всём
new_new_longlist = []
for i in tqdm(range(len(new_longlist))):
    a = new_longlist[i]
    new_new_longlist.append(tuple((a[0], no_punctuation(a[1]))))

In [222]:
# Крутой уровень вложенности получился, ничего не скажешь!
new_new_longlist[1]

(2,
 'представлять товарищ жук сегодня сон присниться говорить сталин москва минск нападывать киев канцлер германия умолять война прекращать')

### Часть 2.4. Запись в БД

Запишем всё, что получилось в БД. (немного неэлегантно, зато результативно)

In [237]:
soskok_prognuvshis_query = """
SELECT anecs.id, anecs.text, anecs.time, anecs.likes FROM anecs
"""

In [239]:
conn = sqlite3.connect('le_petit_bd.db')
cur = conn.cursor()
cur.execute(soskok_prognuvshis_query, )
longlist = cur.fetchall()
longlist[0] # Аккуратно, это новый лонглист

(1,
 'Впервые за 28 лет депутатам придётся жить, лечиться, учить своих детей в условиях, которые они все эти годы создавали.',
 2022,
 1406)

Собираем набор инфы в красивейшие туплики

In [252]:
braveness = []
for i in range(len(longlist)):
    k = tuple((longlist[i][1], new_new_longlist[i][1], longlist[i][2], longlist[i][3]))
    braveness.append(k)
braveness[7]

('Правильно ли я понял, что на Казахстан напали террористы и потребовали снизить цены на газ?',
 'правильно понимать казахстан нападывать террорист потребовать снижать цена газ',
 2022,
 1113)

In [253]:
cur.executemany(
                """
                INSERT INTO anecs 
                    (text, lemtext, time, likes)
                    VALUES (?, ?, ?, ?)
                """, (
                    braveness)
            )
conn.commit()
conn.close()

## Комментарий: из-за несовместимости версии питона часть работы будет в гугл колабе

### Комментарий к комментарию: google colab не потянул ОЗУ, потому работала в юпитере, просто другой тетрадке.

# Бонус
### Часть дополнительная.  Рандомный анекдот

Было бы глупо не попробовать. Анекдоты они когда сюрреалистичные - иногда даже смешнее обычных.

Для попытки генерации воспользуюсь **markovify**. 

In [196]:
! pip install markovify

In [197]:
import markovify

Обучаю на своих 6000 анекдотов.

In [194]:
all_anecs = ""
for i in longlist:
    all_anecs += i[1]
    all_anecs += " "

In [198]:
pop_rikolu_model = markovify.Text(all_anecs)

In [200]:
for i in range(5):
    print(i, pop_rikolu_model.make_sentence())

0 МИД РФ анонсировал приезд на переговоры в Израиле.То есть, Путин рассказывал бы двум евреям про денацификацию.
1 Минеральная вода - это экстремизм.
2 Даже женился благодаря ему. - На флоридской вилле Рокфеллера, в его время сажали за анекдоты.
3 Проблема не в этом...
4 А с нормальными людьми на улице проходит мимо меня, прижимая к себе и покупают дипломы Оксфордов и Гарвардов.


Лично я в востороге от **Минеральная вода - это экстремизм.**, и было бы скучно не вставить такую функцию)